In [19]:
import pandas as pd


#load real and test
df_train = pd.read_json("../data/tm_minimal/train_split.json")
df_test = pd.read_json("../data/tm_minimal/test_split.json")

Fields where some test categories are not in the training data set
- first_name
- last_name
- player_id
- pseudonym
- citizenship
- injury

In [20]:
# build up
player_ids = df_train.player_id.tolist()
first_names = df_train.first_name.tolist()
last_names = df_train.last_name.tolist()
pseudonyms = df_train.pseudonym.tolist()
citzenships = df_train.citizenship.tolist()
# go thru real and full player_id_to_names, also pseudonym

injury_mapping = df_train.groupby('injury_category')['injury'].apply(lambda x: list(set(x))).to_dict()
# go thru train and fill category to injury


In [21]:
# get unknown injury
# get unknown citizenships
# need mapping from old id to new[name, lastname, id, pseudonym, citizenship]
taken_ids = []
player_id_to_names = {}

In [22]:
import random
from collections import defaultdict
random.seed(42)

# Create sets for fast lookup
train_static_values = {
    "first_name": set(first_names),
    "last_name": set(last_names),
    "player_id": set(player_ids),
    "pseudonym": set(pseudonyms),
    "citizenship": set(citzenships),
    "injury": set(df_train.injury.tolist()),
}

# Value pools for replacements
replacement_pool = {
    "first_name": first_names,
    "last_name": last_names,
    "player_id": player_ids,
    "pseudonym": pseudonyms,
    "citizenship": citzenships,
}

# Cache to store replacements per test player
player_static_replacements = {}

# iterate over test playerrs
for pid in df_test.player_id.unique():
    player_rows = df_test[df_test.player_id == pid]
    sample_row = player_rows.iloc[0]
    replacement = {}

    # check for each value if it is in the training set
    for field in ["player_id", "first_name", "last_name", "pseudonym", "citizenship"]:
        val = sample_row[field]
        if val not in train_static_values[field]:
            replacement[field] = random.choice(replacement_pool[field])
        else: # is in the trainnig set: can keep old value
            replacement[field] = val

    player_static_replacements[pid] = replacement


# change values in test set with possibly replaced field values
def apply_replacements(row):
    repl = player_static_replacements[row.player_id]
    for field in ["player_id", "first_name", "last_name", "pseudonym", "citizenship"]:
        row[field] = repl[field]
    return row

df_test = df_test.apply(apply_replacements, axis=1)



def replace_injury(row):
    if row["injury"] not in train_static_values["injury"]:
        options = injury_mapping.get(row["injury_category"], [])
        if options:
            return random.choice(options)
    return row["injury"]

df_test["injury"] = df_test.apply(replace_injury, axis=1)


In [24]:
df_test.to_json("../data/tm_minimal/test_split_no_new_categories.json", orient="records")
df_test.shape

(7630, 45)